In [2]:
import guidance

In [3]:
guidance.llm = guidance.llms.OpenAI("text-davinci-003")

# Basic templating

Single variable


In [4]:
prompt = guidance('''What is {{example}}?''')
prompt

Stop program What is {{example}} ?

In [5]:
prompt(example='truth')

Stop program What is truth ?

Objects

In [6]:
people = ['John', 'Mary', 'Bob', 'Alice']
ideas = [{'name': 'truth', 'description': 'the state of being the case'},
         {'name': 'love', 'description': 'a strong feeling of affection'},]
prompt = guidance('''List of people:
{{#each people}}- {{this}}
{{~! This is a comment. The ~ removes adjacent whitespace either before or after a tag, depending on where you place it}}
{{/each~}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')
prompt(people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Including prompts

In [17]:
prompt1 = guidance('''List of people:
{{#each people}}- {{this}}
{{/each~}}''')
prompt2 = guidance('''{{>prompt1}}
List of ideas:
{{#each ideas}}{{this.name}}: {{this.description}}
{{/each}}''')
prompt2(prompt1=prompt1, people=people, ideas=ideas)

Stop program List of people:
 - John 
- Mary 
- Bob 
- Alice 
 
List of ideas:
 truth : the state of being the case 
 love : a strong feeling of affection

Generating

In [16]:
prompt = guidance('''The best thing about the beach is {{~gen 'best' temperature=0.7 max_tokens=7}}''')
prompt = prompt()
prompt

Stop program The best thing about the beach is that it is a place to relax

Flushing caches

In [ ]:
guidance.llms.OpenAI.cache.clear()

16

Select

In [30]:
prompt = guidance('''Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: {{example}}
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{or}} Maybe{{/select}}''')
prompt = prompt(example='I hate tacos')
prompt


Stop program Is the following sentence offensive? Please answer with a single word, either "Yes", "No", or "Maybe".
Sentence: I hate tacos 
Answer: Maybe

In [28]:
prompt['logprobs']

{' Yes': -1.5689583, ' No': -7.332395, ' Maybe': -0.23746304}

Multiple generates in sequence

In [31]:
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')
prompt = prompt(email='I hate tacos')
prompt

Stop program Generate a response to the following email:
 I hate tacos .
Response: That's too bad! Tacos are one of my favorite meals. 

Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

In [34]:
prompt['response'], prompt['answer']

(" That's too bad! Tacos are one of my favorite meals.", ' No')

Hidden

In [35]:
prompt = guidance('''{{#block hidden=True}}Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}{{/block}}
I will show you an email and a response, and you will tell me if it's offensive.
Email: {{email}}.
Response: {{response}}
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}''')
prompt = prompt(email='I hate tacos')
prompt


Stop program {{!--GHIDDEN:Generate a response to the following email:
{{!--GMARKER_START_variable_ref$&#123;&#123;email&#125;&#125;$--_END_ENDI hate tacos{{!--GMARKER_END_variable_ref$$--_END_END.
Response: 
I will show you an email and a response, and you will tell me if it's offensive.
Email: I hate tacos .
Response: That's too bad! Tacos are one of my favorite meals. 
Is the response above offensive in any way? Please answer with a single word, either "Yes" or "No".
Answer: No

Generating with `n>1`

In [39]:
prompt = guidance('''The best thing about the beach is {{~gen 'best' n=3 temperature=0.7 max_tokens=7}}''')
prompt = prompt()
prompt['best']

[' that it is a great place to',
 ' being able to relax in the sun',
 " that it's a great place to"]

Calling Functions

In [42]:
def aggregate(best):
    return '\n'.join(['- ' + x for x in best])
prompt = guidance('''The best thing about the beach is {{~gen 'best' n=3 temperature=0.7 max_tokens=7 hidden=True}}
{{aggregate best}}''')
prompt = prompt(aggregate=aggregate)
prompt

Stop program The best thing about the beach is 
 - the peaceful atmosphere and the beautiful scenery
- the feeling of relaxation and peace you
- the feeling of sand between your toes

In [12]:
a['logprobs']

{' Yes': -0.0006238507, ' No': -100}

Await

In [ ]:
prompt = guidance('''Generate a response to the following email:
{{email}}.
Response:{{gen "response"}}
{{await 'instruction'}}
{{gen 'updated_response'}}''', stream=True)
prompt = prompt(email='Hello there')
prompt


Stop program Generate a response to the following email:
 Hello there .
Response: Hi! How can I help you? 
 {{await 'instruction'}} 
 {{gen 'updated_response'}}

In [ ]:
prompt2 = prompt(instruction='Please translate the response above to Portuguese.')
prompt2

Stop program Generate a response to the following email:
Hello there.
Response: Hi! How can I help you?
 Please translate the response above to Portuguese. 
 Olá! Como posso ajudar?

In [47]:
prompt2 = prompt(instruction='Please translate the response above to Chinese.')
prompt2

Stop program Generate a response to the following email:
Hello there.
Response: Hi! How can I help you?
 Please translate the response above to Chinese. 
 你好！我能为你做些什么？

## Chat

Chat backend, chat_completion=True

In [9]:
guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)


In [10]:
prompt = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
{{conversation_question}}<|im_end|>
<im_start|>assistant
{{gen 'response'}}<|im_end|>''', stream=True)
prompt(conversation_question='What is the meaning of life?')

Stop program <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
 What is the meaning of life? <|im_end|>
<im_start|>assistant
 The meaning of life is a philosophical question and has been debated for centuries. Different people, cultures, and belief systems hold various interpretations of the meaning of life. Some believe it is to find happiness, personal fulfillment, or to contribute positively to society. Others may find meaning through spiritual or religious beliefs. Ultimately, the meaning of life is subjective and can vary from person to person. <|im_end|>

Multistep

In [17]:

experts = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.<|im_end|>
<|im_start|>assistant
{{gen 'experts' stop="<|im_end|>" temperature=0 max_tokens=300}}<|im_end|>
<|im_start|>user
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:<|im_end|>
<|im_start|>assistant
{{gen 'answer' stop="<|im_end|>" temperature=0 max_tokens=500}}<|im_end|>''', stream=True)
experts(query='What is the meaning of life?')

Stop program

With hidden

In [31]:
prompt = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|>
{{#block hidden=True}}<|im_start|>user
Please tell me a joke<|im_end|>
<im_start|>assistant
{{gen 'joke'}}<|im_end|>{{/block}}<|im_start|>user
Is the following joke funny? Why or why not?
{{joke}}<|im_end|>
<|im_start|>assistant
{{gen 'funny'}}<|im_end|>''', stream=True)
p = prompt()


In [32]:
p

Stop program <|im_start|>system
You are a helpful assistant.<|im_end|>
 {{!--GHIDDEN:<|im_start|>user
Please tell me a joke<|im_end|>
<im_start|>assistant
{{!--GMARKER_START_gen$&#123;&#123;gen 'joke'&#125;&#125;$--_END_ENDWhy did the chicken cross the playground?
To get to the other slide! <|im_start|>user
Is the following joke funny? Why or why not?
 Why did the chicken cross the playground?
To get to the other slide! <|im_end|>
<|im_start|>assistant
 That is a funny joke! <|im_end|>

Agents

In [20]:
prompt = guidance('''<|im_start|>system
You are a helpful assistant<|im_end|>
{{#geneach 'conversation'}}
<|im_start|>user
{{set 'this.user_text' (await 'user_text')}}<|im_end|>
<|im_start|>assistant
{{gen 'this.ai_text' stop="<|im_end|>" temperature=0 max_tokens=300}}<|im_end|>{{/geneach}}''', echo=True)
prompt= prompt(user_text ='hi there')
prompt['conversation']

Stop program <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
I want a response to the following question:
 What is the meaning of life? 
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.<|im_end|>
<|im_start|>assistant
 The three world-class experts who would be great at answering this question are: 
1. Stephen Hawking 
2. Albert Einstein 
3. Plato. <|im_end|>
<|im_start|>user
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:<|im_end|>
<|im_start|>assistant
 ANSWER: The meaning of life is a complex and subjective concept that is open to interpretation. Some might argue that the meaning of life is to find joy and fulfillment in life's experiences, while others might argue that the meaning of life is to find a purpose and use it to make a positive impact on the world. Ultimately, the meaning of life is a personal journey that each individual must discover for themselves. <|im_end|>

In [21]:
prompt = prompt(user_text = 'what is the meaning of life?')
prompt

Stop program <|im_start|>system
You are a helpful assistant<|im_end|>

<|im_start|>user
hi there<|im_end|>
<|im_start|>assistant
Hello! How can I help you?<|im_end|>
<|im_start|>user
 what is the meaning of life? <|im_end|>
<|im_start|>assistant
 That is a difficult question to answer. Everyone has their own interpretation of what the meaning of life is. What do you think it is? <|im_end|>
<|im_start|>user
 {{set 'conversation[-1].user_text' (await 'user_text')}} <|im_end|>
<|im_start|>assistant
 {{gen 'conversation[-1].ai_text' stop="<|im_end|>" temperature=0 max_tokens=300}} <|im_end|> {{#geneach 'conversation'}} 
<|im_start|>user
 {{set 'this.user_text' (await 'user_text')}} <|im_end|>
<|im_start|>assistant
 {{gen 'this.ai_text' stop="<|im_end|>" temperature=0 max_tokens=300}} <|im_end|> {{/geneach}}

Using tools

In [39]:
str(s)

'Seach results:\n<result>\nOpenAI - Wikipedia\nOpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.\n</result>\n<result>\nAbout - OpenAI\nAbout OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI\n</result>\n<result>\nIlya Sutskever | Stanford HAI\nIlya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and 

In [47]:
def is_search(completion):
    return '<search>' in completion
def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
        'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
        {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
        'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
        {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
        'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]
search_demo = guidance('''Seach results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}''')
demo_results = [
    {'title': 'OpenAI - Wikipedia', 'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
    {'title': 'About - OpenAI', 'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'}, 
    {'title': 'Ilya Sutskever | Stanford HAI', 'snippet': '''Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.'''}
]
s = search_demo(results=demo_results)
practice_round = [
    {'role': 'user', 'content' : 'Who are the founders of OpenAI?'},
    {'role': 'assistant', 'content': '<search>Who are the founders of OpenAI</search>'},
    {'role': 'user', 'content': str(search_demo(results=demo_results))},
    {'role': 'assistant', 'content': 'The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.'},
]
prompt = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.<|im_end|>
<|im_start|>assistant
Ok, I will do that. Let's do a practice round<|im_end|>

{{~#each practice}}
<|im_start|>{{this.role}}
{{this.content}}<|im_end|>
{{~/each~}}

<|im_start|>user
That was great, now let's do another one.<|im_end|>
<|im_start|>assistant
Sounds good<|im_end|>
<|im_start|>user
{{user_query}}<|im_end|>
<|im_start|>assistant
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search><|im_end|>
<|im_start|>user
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}<|im_end|>
<|im_start|>assistant
{{gen "answer"}}
<|im_end|>
{{/if}}''')
query = "What is Facebook's stock price right now?"
prompt = prompt(user_query=query, search=search, is_search=is_search, practice=practice_round)
prompt


Stop program <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.<|im_end|>
<|im_start|>assistant
Ok, I will do that. Let's do a practice round<|im_end|> 
<|im_start|> user 
 Who are the founders of OpenAI? <|im_end|>
<|im_start|> assistant 
 <search>Who are the founders of OpenAI</search> <|im_end|>
<|im_start|> user 
 Seach results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result> <|im_end|>
<|im_start|> assistant 
 The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others. <|im_end|> 
<|im_start|>user
That was great, now let's do another one.<|im_end|>
<|im_start|>assistant
Sounds good<|im_end|>
<|im_start|>user
 What is Facebook's stock price right now? <|im_end|>
<|im_start|>assistant
 <search>Facebook's stock price right now <span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;</search><|im_end|>
<|im_start|>user
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;<|im_end|>
<|im_start|>assistant
&#123;&#123;gen "answer"&#125;&#125;
<|im_end|>
&#123;&#123;/if&#125;&#125;'></search><|im_end|>
<|im_start|>user
Search results: 
<result>
 How do I cancel a Subscription? | Facebook Help Center 
 To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ... 
</result>
<result>
 News | FACEBOOK Stock Price Today | Analyst Opinions - Insider 
 Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr... 
</result>
<result>
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ... 
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved 
</result> <|im_end|>
<|im_start|>assistant
 Facebook's stock price right now is $197.81. Please note that stock prices are constantly changing, so this value might not be accurate at the time you check it. 
<|im_end|>

In [11]:

def is_search(completion):
    return '<search>' in completion
def search(query):
    # Fake search results
    return [{'title': 'How do I cancel a Subscription? | Facebook Help Center',
  'snippet': "To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ..."},
 {'title': 'News | FACEBOOK Stock Price Today | Analyst Opinions - Insider',
  'snippet': 'Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr...'},
 {'title': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ...',
  'snippet': 'Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved'}]
prompt = guidance('''<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.<|im_end|>
<|im_start|>assistant
Ok, I will do that. Let's do a practice round<|im_end|>
<|im_start|>user
Who are the founders of OpenAI?<|im_end|>
<|im_start|>assistant
<search>Who are the founders of OpenAI</search><|im_end|>
<|im_start|>user
Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result><|im_end|>
<|im_start|>assistant
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.<|im_end|>
<|im_start|>user
That was great, now let's do another one.<|im_end|>
<|im_start|>assistant
Sounds good<|im_end|>
<|im_start|>user
{{user_query}}<|im_end|>
<|im_start|>assistant
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search><|im_end|>
<|im_start|>user
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}<|im_end|>
<|im_start|>assistant
{{gen "answer"}}
<|im_end|>
{{/if}}''')
query = "What is Facebook's stock price right now?"
prompt = prompt(user_query=query, search=search, is_search=is_search)
prompt

Stop program <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.<|im_end|>
<|im_start|>assistant
Ok, I will do that. Let's do a practice round<|im_end|>
<|im_start|>user
Who are the founders of OpenAI?<|im_end|>
<|im_start|>assistant
<search>Who are the founders of OpenAI</search><|im_end|>
<|im_start|>user
Search results:
<result>
OpenAI - Wikipedia
OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.
</result>
<result>
About - OpenAI
About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI
</result>
<result>
Ilya Sutskever | Stanford HAI
Ilya Sutskever is Co-founder and Chief Scientist of OpenAI, which aims to build artificial general intelligence that benefits all of humanity. He leads research at OpenAI and is one of the architects behind the GPT models. Prior to OpenAI, Ilya was co-inventor of AlexNet and Sequence to Sequence Learning.
</result><|im_end|>
<|im_start|>assistant
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.<|im_end|>
<|im_start|>user
That was great, now let's do another one.<|im_end|>
<|im_start|>assistant
Sounds good<|im_end|>
<|im_start|>user
 What is Facebook's stock price right now? <|im_end|>
<|im_start|>assistant
 <search>Facebook's stock price right now <span style='background-color: rgba(0, 138.56128016, 250.76166089, 0.25); display: inline;' title='&#123;&#123;#if (is_search query)&#125;&#125;</search><|im_end|>
<|im_start|>user
Search results: &#123;&#123;#each (search query)&#125;&#125;
<result>
&#123;&#123;this.title&#125;&#125;
&#123;&#123;this.snippet&#125;&#125;
</result>&#123;&#123;/each&#125;&#125;<|im_end|>
<|im_start|>assistant
&#123;&#123;gen "answer"&#125;&#125;
<|im_end|>
&#123;&#123;/if&#125;&#125;'></search><|im_end|>
<|im_start|>user
Search results: 
<result>
 How do I cancel a Subscription? | Facebook Help Center 
 To stop a monthly Subscription to a creator: Go to the creator's Facebook Page using the latest version of the Facebook app for iOS, Android or from a computer. Select Go to Supporter Hub. Select . Select Manage Subscription to go to the iTunes or Google Play Store and cancel your subscription. Cancel your Subscription at least 24 hours before ... 
</result>
<result>
 News | FACEBOOK Stock Price Today | Analyst Opinions - Insider 
 Stock | News | FACEBOOK Stock Price Today | Analyst Opinions | Markets Insider Markets Stocks Indices Commodities Cryptocurrencies Currencies ETFs News Facebook Inc (A) Cert Deposito Arg Repr... 
</result>
<result>
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart ... 
 Facebook Stock Price Today (NASDAQ: META) Quote, Market Cap, Chart | WallStreetZen Meta Platforms Inc Stock Add to Watchlist Overview Forecast Earnings Dividend Ownership Statistics $197.81 +2.20 (+1.12%) Updated Mar 20, 2023 Meta Platforms shares are trading... find out Why META Price Moved with a free WallStreetZen account Why Price Moved 
</result> <|im_end|>
<|im_start|>assistant
 Facebook's stock price right now is $197.81. Please note that stock prices are constantly changing, so this value might not be accurate at the time you check it. 
<|im_end|>